# Transfer Learning CIFAR10

* Train a simple convnet on the CIFAR dataset the first 5 output classes [0..4].
* Freeze convolutional layers and fine-tune dense layers for the last 5 ouput classes [5..9].


### 1. Import CIFAR10 data and create 2 datasets with one dataset having classes from 0 to 4 and other having classes from 5 to 9 

In [1]:
from keras.datasets import cifar10
import numpy as np
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense,Conv2D,MaxPool2D,Flatten,Dropout
from keras.optimizers import Adam

Using TensorFlow backend.


In [2]:
(xtrain,ytrain),(xtest,ytest)=cifar10.load_data()


170500096/170498071 [==============================] - 7s 0us/step


In [3]:
xtrain.shape,ytrain.shape,xtest.shape,ytest.shape

((50000, 32, 32, 3), (50000, 1), (10000, 32, 32, 3), (10000, 1))

In [0]:
ytrain=ytrain.flatten()
ytest=ytest.flatten()

In [0]:
x_train_lt5 = xtrain[ytrain < 5]
y_train_lt5 = ytrain[ytrain < 5]
x_test_lt5 = xtest[ytest < 5]
y_test_lt5 = ytest[ytest < 5]

x_train_gt5 = xtrain[ytrain >= 5]
y_train_gt5 = ytrain[ytrain >= 5] - 5  # make classes start at 0 for
x_test_gt5 = xtest[ytest >= 5]         # np_utils.to_categorical
y_test_gt5 = ytest[ytest >= 5] - 5

### 2. Use One-hot encoding to divide y_train and y_test into required no of output classes

In [0]:
y_train_lt5=to_categorical(y_train_lt5,num_classes=5)
y_test_lt5=to_categorical(y_test_lt5,num_classes=5)

In [0]:
y_train_gt5=to_categorical(y_train_gt5,num_classes=5)
y_test_gt5=to_categorical(y_test_gt5,num_classes=5)

In [0]:
x_train_lt5=(x_train_lt5/255).astype('float32')
x_test_lt5=(x_test_lt5/255).astype('float32')

In [0]:
x_train_gt5=(x_train_gt5/255).astype('float32')
x_test_gt5=(x_test_gt5/255).astype('float32')

### 3. Build a sequential neural network model which can classify the classes 0 to 4 of CIFAR10 dataset with at least 80% accuracy on test data

In [10]:
model =Sequential()
model.add(Conv2D(64,(3,3),input_shape=(32,32,3),activation='relu'))
model.add(Conv2D(32,(3,3),activation='relu'))
model.add(MaxPool2D((2,2)))
model.add(Dropout(0.20))

model.add(Conv2D(64,(3,3),activation='relu'))
model.add(Conv2D(32,(3,3),activation='relu'))
model.add(MaxPool2D((2,2)))
model.add(Dropout(0.20))

model.add(Flatten())
model.add(Dense(128,activation='relu'))

model.add(Dense(5,activation='softmax'))
model.compile('adam',metrics=['acc'],loss='categorical_crossentropy')
model.summary()

W0710 15:03:48.425228 140121292216192 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0710 15:03:48.462348 140121292216192 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0710 15:03:48.470194 140121292216192 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0710 15:03:48.506686 140121292216192 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:3976: The name tf.nn.max_pool is deprecated. Please use tf.nn.max_pool2d instead.

W0710 15:03:48.510549 140121292216192 deprecation_wrapp

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 30, 30, 64)        1792      
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 28, 28, 32)        18464     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 14, 14, 32)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 14, 14, 32)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 12, 12, 64)        18496     
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 10, 10, 32)        18464     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 5, 5, 32)          0         
__________

In [11]:
model.fit(x_train_lt5,y_train_lt5,batch_size=1500,epochs=50,validation_data=(x_test_lt5,y_test_lt5))

W0710 15:03:50.737560 140121292216192 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Train on 25000 samples, validate on 5000 samples
Epoch 1/50
25000/25000 [==============================] - 11s 428us/step - loss: 1.4439 - acc: 0.3703 - val_loss: 1.2232 - val_acc: 0.4974
Epoch 2/50
25000/25000 [==============================] - 2s 78us/step - loss: 1.1910 - acc: 0.5111 - val_loss: 1.1002 - val_acc: 0.5562
Epoch 3/50
25000/25000 [==============================] - 2s 78us/step - loss: 1.1017 - acc: 0.5448 - val_loss: 1.0403 - val_acc: 0.5730
Epoch 4/50
25000/25000 [==============================] - 2s 78us/step - loss: 1.0366 - acc: 0.5708 - val_loss: 0.9914 - val_acc: 0.5860
Epoch 5/50
25000/25000 [==============================] - 2s 79us/step - loss: 0.9997 - acc: 0.5893 - val_loss: 0.9638 - val_acc: 0.6050
Epoch 6/50
25000/25000 [==============================] - 2s 79us/step - loss: 0.9655 - acc: 0.6088 - val_loss: 0.9111 - val_acc: 0.6302
Epoch 7/50
25000/25000 [==============================] - 2s 79us/step - loss: 0.9321 - acc: 0.6202 - val_loss: 0.9070 - val_ac

### 4. In the model which was built above (for classification of classes 0-4 in CIFAR10), make only the dense layers to be trainable and conv layers to be non-trainable

In [12]:
for layers in model.layers:
    print(layers.name)
    if('dense' not in layers.name):
        layers.trainable = False
        print(layers.name + 'is not trainable\n')
    if('dense' in layers.name):
        print(layers.name + ' is trainable\n')

conv2d_1
conv2d_1is not trainable

conv2d_2
conv2d_2is not trainable

max_pooling2d_1
max_pooling2d_1is not trainable

dropout_1
dropout_1is not trainable

conv2d_3
conv2d_3is not trainable

conv2d_4
conv2d_4is not trainable

max_pooling2d_2
max_pooling2d_2is not trainable

dropout_2
dropout_2is not trainable

flatten_1
flatten_1is not trainable

dense_1
dense_1 is trainable

dense_2
dense_2 is trainable



### 5. Utilize the the model trained on CIFAR 10 (classes 0 to 4) to classify the classes 5 to 9 of CIFAR 10  (Use Transfer Learning) <br>
Achieve an accuracy of more than 85% on test data

In [13]:
model.fit(x_train_gt5,y_train_gt5,validation_data=(x_test_gt5,y_test_gt5),batch_size=32,epochs=50)

Train on 25000 samples, validate on 5000 samples
Epoch 1/50
  480/25000 [..............................] - ETA: 9s - loss: 2.1287 - acc: 0.2125 

/usr/local/lib/python3.6/dist-packages/keras/engine/training.py:490: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


25000/25000 [==============================] - 6s 228us/step - loss: 1.0945 - acc: 0.5521 - val_loss: 0.7488 - val_acc: 0.7084
Epoch 2/50
25000/25000 [==============================] - 5s 215us/step - loss: 0.6846 - acc: 0.7400 - val_loss: 0.6554 - val_acc: 0.7544
Epoch 3/50
25000/25000 [==============================] - 5s 214us/step - loss: 0.5579 - acc: 0.7920 - val_loss: 0.4778 - val_acc: 0.8224
Epoch 4/50
25000/25000 [==============================] - 5s 212us/step - loss: 0.4781 - acc: 0.8258 - val_loss: 0.4893 - val_acc: 0.8224
Epoch 5/50
25000/25000 [==============================] - 5s 213us/step - loss: 0.4201 - acc: 0.8448 - val_loss: 0.3948 - val_acc: 0.8590
Epoch 6/50
25000/25000 [==============================] - 5s 217us/step - loss: 0.3785 - acc: 0.8612 - val_loss: 0.4009 - val_acc: 0.8570
Epoch 7/50
25000/25000 [==============================] - 5s 215us/step - loss: 0.3494 - acc: 0.8732 - val_loss: 0.3621 - val_acc: 0.8690
Epoch 8/50
25000/25000 [=====================

In [14]:
model.evaluate(x_test_gt5,y_test_gt5)

5000/5000 [==============================] - 0s 98us/step


[0.4231841671898961, 0.9004]

## Sentiment analysis <br> 

The objective of the second problem is to perform Sentiment analysis from the tweets data collected from the users targeted at various mobile devices.
Based on the tweet posted by a user (text), we will classify if the sentiment of the user targeted at a particular mobile device is positive or not.

### 6. Read the dataset (tweets.csv) and drop the NA's while reading the dataset

In [0]:
import pandas as pd
data = pd.read_csv('./tweets.csv', encoding = "ISO-8859-1").dropna()

In [18]:
data.shape

(3291, 3)

In [19]:
data.head()

,tweet_text,emotion_in_tweet_is_directed_at,is_there_an_emotion_directed_at_a_brand_or_product
0,.@wesley83 I have a 3G iPhone. After 3 hrs twe...,iPhone,Negative emotion
1,@jessedee Know about @fludapp ? Awesome iPad/i...,iPad or iPhone App,Positive emotion
2,@swonderlin Can not wait for #iPad 2 also. The...,iPad,Positive emotion
3,@sxsw I hope this year's festival isn't as cra...,iPad or iPhone App,Negative emotion
4,@sxtxstate great stuff on Fri #SXSW: Marissa M...,Google,Positive emotion


### Consider only rows having Positive emotion and Negative emotion and remove other rows from the dataframe.

In [0]:
data = data[(data['is_there_an_emotion_directed_at_a_brand_or_product'] == 'Positive emotion') | (data['is_there_an_emotion_directed_at_a_brand_or_product'] == 'Negative emotion')]

In [21]:
data.shape

(3191, 3)

In [22]:
data.head()


,tweet_text,emotion_in_tweet_is_directed_at,is_there_an_emotion_directed_at_a_brand_or_product
0,.@wesley83 I have a 3G iPhone. After 3 hrs twe...,iPhone,Negative emotion
1,@jessedee Know about @fludapp ? Awesome iPad/i...,iPad or iPhone App,Positive emotion
2,@swonderlin Can not wait for #iPad 2 also. The...,iPad,Positive emotion
3,@sxsw I hope this year's festival isn't as cra...,iPad or iPhone App,Negative emotion
4,@sxtxstate great stuff on Fri #SXSW: Marissa M...,Google,Positive emotion


### 7. Represent text as numerical data using `CountVectorizer` and get the document term frequency matrix

#### Use `vect` as the variable name for initialising CountVectorizer.

In [0]:
from sklearn.feature_extraction.text import CountVectorizer
vect=CountVectorizer()


In [0]:
tweet_text=vect.fit(data['tweet_text'])

### 8. Find number of different words in vocabulary

In [25]:
l=len(tweet_text.get_feature_names())
print("total no of different words",l)

total no of different words 5648


#### Tip: To see all available functions for an Object use dir

### Find out how many Positive and Negative emotions are there.

Hint: Use value_counts on that column

In [27]:
pd.value_counts(data['is_there_an_emotion_directed_at_a_brand_or_product'])

Positive emotion    2672
Negative emotion     519
Name: is_there_an_emotion_directed_at_a_brand_or_product, dtype: int64

###  Change the labels for Positive and Negative emotions as 1 and 0 respectively and store in a different column in the same dataframe named 'label'

Hint: use map on that column and give labels

In [0]:
data['label'] = data.is_there_an_emotion_directed_at_a_brand_or_product.map({'Positive emotion':1, 'Negative emotion':0})

### 9. Define the feature set (independent variable or X) to be `text` column and `labels` as target (or dependent variable)  and divide into train and test datasets

In [0]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(data['tweet_text'],data['label'],test_size=0.20,random_state=1)

## 10. **Predicting the sentiment:**


### Use Naive Bayes and Logistic Regression and their accuracy scores for predicting the sentiment of the given text

In [0]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn import metrics

## 11. Create a function called `tokenize_predict` which can take count vectorizer object as input and prints the accuracy for x (text) and y (labels)

In [0]:
def tokenize_test(vect):
    x_train_dtm = vect.fit_transform(x_train)
    print('Features: ', x_train_dtm.shape[1])
    x_test_dtm = vect.transform(x_test)
    nb = MultinomialNB()
    lg=LogisticRegression(solver='lbfgs')
    nb.fit(x_train_dtm, y_train)
    lg.fit(x_train_dtm, y_train)
    y_pred_class_nb = nb.predict(x_test_dtm)
    y_pred_class_lg = lg.predict(x_test_dtm)
    print('Accuracy when use Navie bayes ', metrics.accuracy_score(y_test, y_pred_class_nb))
    print('Accuracy when use Logistic regression ', metrics.accuracy_score(y_test, y_pred_class_lg))

### Create a count vectorizer function which includes n_grams = 1,2  and pass it to tokenize_predict function to print the accuracy score

In [32]:
# include 1-grams and 2-grams
vect = CountVectorizer(ngram_range=(1, 2))
tokenize_test(vect)

Features:  25815
Accuracy when use Navie bayes  0.838810641627543
Accuracy when use Logistic regression  0.8544600938967136


### 12. Create a count vectorizer function with stopwords = 'english'  and pass it to tokenize_predict function to print the accuracy score

In [33]:
vect=CountVectorizer(stop_words='english')
tokenize_test(vect)

Features:  4796
Accuracy when use Navie bayes  0.838810641627543
Accuracy when use Logistic regression  0.8513302034428795


### 13. Create a count vectorizer function with stopwords = 'english' and max_features =300  and pass it to tokenize_predict function to print the accuracy score

In [34]:
vect=CountVectorizer(stop_words='english',max_features=300)
tokenize_test(vect)

Features:  300
Accuracy when use Navie bayes  0.7996870109546166
Accuracy when use Logistic regression  0.8231611893583725


### 14. Create a count vectorizer function with n_grams = 1,2  and max_features = 15000  and pass it to tokenize_predict function to print the accuracy score

In [35]:
vect=CountVectorizer(ngram_range=(1, 2),max_features=15000)
tokenize_test(vect)

Features:  15000
Accuracy when use Navie bayes  0.8341158059467919
Accuracy when use Logistic regression  0.8622848200312989


### 15. Create a count vectorizer function with n_grams = 1,2  and include terms that appear at least 2 times (min_df = 2)  and pass it to tokenize_predict function to print the accuracy score

In [36]:
vect=CountVectorizer(ngram_range=(1, 2),min_df=2)
tokenize_test(vect)

Features:  8298
Accuracy when use Navie bayes  0.8435054773082942
Accuracy when use Logistic regression  0.8622848200312989
